<a href="https://colab.research.google.com/github/haziqzikry/business-intelligence-analytics-project/blob/main/0_amazon_reviews_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 23.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [2]:
from datasets import load_dataset
import pandas as pd
from tqdm import tqdm
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Chunking Method


In [4]:
# Function to process and clean a chunk of data
def process_chunk(chunk, category):
    chunk = pd.DataFrame(chunk)

    # Convert 'timestamp' column to datetime and filter by 2022-2023
    if 'timestamp' in chunk.columns:
        chunk['timestamp'] = pd.to_datetime(chunk['timestamp'], unit='ms')
        chunk = chunk[(chunk['timestamp'] >= '2023-01-01') & (chunk['timestamp'] <= '2023-12-31')]

    # Remove the 'images' column
    if 'images' in chunk.columns:
        chunk = chunk.drop(columns=['images'])

    # Rename columns
    chunk = chunk.rename(columns={
        'title': 'review_title',
        'text': 'review_text',
        'timestamp': 'date'
    })

    # Drop missing values
    chunk = chunk.dropna()

    # Add 'Category' column
    chunk['main_category'] = category.replace('_', ' ')


    return chunk

In [5]:
# Function to load and clean dataset in chunks
def load_and_clean_category_in_chunks(category, chunk_size=100000):
    # Load the dataset
    dataset = load_dataset(
        "McAuley-Lab/Amazon-Reviews-2023",
        f"raw_review_{category}",
        trust_remote_code=True,
    )

    # Determine available splits
    available_splits = list(dataset.keys())
    print(f"Available splits for {category}: {available_splits}")

    # Use the first available split (assuming it has data)
    split_name = available_splits[0]

    # Initialize an empty list to store processed chunks
    processed_chunks = []

    # Process the dataset in chunks with progress bar
    for start in tqdm(range(0, len(dataset[split_name]), chunk_size), desc=f"Processing {category}"):
        end = start + chunk_size
        chunk = dataset[split_name][start:end]
        processed_chunk = process_chunk(chunk, category)
        processed_chunks.append(processed_chunk)

        # Save intermediate chunk to CSV to avoid losing progress
        processed_chunk.to_csv(f'processed_chunk_{category}_{start}_{end}.csv', index=False)
        print(f"Processed and saved chunk {start} to {end}")

    # Combine all processed chunks into a single DataFrame
    combined_df = pd.concat(processed_chunks, ignore_index=True)

    # Save the combined processed dataset to a CSV file
    combined_df.to_csv(f'{category}__reviews_processed_combined.csv', index=False)

    return combined_df



In [ ]:
# Example usage
df = load_and_clean_category_in_chunks("Toys_and_Games")


In [ ]:
_ = df.sample(50000)

In [ ]:
_.shape

(50000, 10)

In [ ]:
_['date'].min(), _['date'].max()

(Timestamp('2023-01-01 00:00:54.068000'),
 Timestamp('2023-09-12 09:25:48.128000'))

In [ ]:
# Export the cleaned dataframes to CSV files in Google Drive
drive_path = '/content/drive/MyDrive/Amazon_Reviews_2023/Reviews/'
_.to_csv(drive_path + f"Sports_and_Outdoors_reviews_processed.csv", index=False)

In [ ]:
# Define the categories
categories = [
    # "Digital_Music", - DONE
    # "Magazine_Subscriptions", - DONE
    # "Movies_and_TV",
    # "Musical_Instruments", - DONE
    "Sports_and_Outdoors"
    # "Toys_and_Games",
    # "Video_Games" - DONE
]

In [ ]:
# Function to load, clean, and transform the dataset
def load_and_clean_category(category):
    # Load the dataset
    dataset = load_dataset(
        "McAuley-Lab/Amazon-Reviews-2023",
        f"raw_review_{category}",
        trust_remote_code=True,
    )

    # Determine available splits
    available_splits = list(dataset.keys())
    print(f"Available splits for {category}: {available_splits}")

    # Use the first available split (assuming it has data)
    split_name = available_splits[0]

    # Convert to pandas DataFrame
    df = pd.DataFrame(dataset[split_name])

    # Convert 'timestamp' column to datetime and filter by 2022-2023
    if 'timestamp' in df.columns:
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df = df[(df['timestamp'] >= '2022-01-01') & (df['timestamp'] <= '2023-12-31')]
        # df['timestamp'] = df['timestamp'].dt.date  # Show only the date part

    # Remove the 'images' column
    if 'images' in df.columns:
        df = df.drop(columns=['images'])

    # Rename columns
    df = df.rename(columns={
        'title': 'review_title',
        'text': 'review_text',
        'timestamp': 'date'
    })

    # Drop missing values
    df = df.dropna()

    # Add 'Category' column
    df['main_category'] = category.replace('_', ' ')

    return df

In [ ]:
# Load and clean datasets for all categories with a progress bar
category_dataframes = {}
for category in tqdm(categories, desc="Loading and cleaning datasets"):
    category_dataframes[category] = load_and_clean_category(category)

Loading and cleaning datasets:   0%|          | 0/1 [00:00<?, ?it/s]

Generating full split: 0 examples [00:00, ? examples/s]

Available splits for Sports_and_Outdoors: ['full']


In [ ]:
# # Export the cleaned dataframes to CSV files
# for category, df in category_dataframes.items():
#     df.to_csv(f"{category}_reviews_processed.csv", index=False)

In [ ]:
# Export the cleaned dataframes to CSV files in Google Drive
drive_path = '/content/drive/MyDrive/Amazon_Reviews_2023/Reviews/'
for category, df in category_dataframes.items():
    df.to_csv(drive_path + f"{category}_reviews_processed.csv", index=False)

In [ ]:
# # Example: Access the cleaned dataframe for Digital_Music
df_test = category_dataframes['Subscription_Boxes']
df_test


,rating,review_title,review_text,asin,parent_asin,user_id,date,helpful_vote,verified_purchase,main_category
5,5.0,Cat toys,My cats used to love them,B07G584SHG,B09WC47S3V,AFMXU7O2HQESL2TVDORFT7I4RERA,2022-04-25 22:51:53.277,2,True,Subscription Boxes
15,2.0,Not too much variet from a at's point of iew.,The toys were generally designed to appeal to ...,B07G4ZWJB6,B09WC47S3V,AFCY4QBVMPNTVS5CJDWJQ56CWU4Q,2023-01-30 04:03:34.404,0,True,Subscription Boxes
20,1.0,Too Expensive,It’s too expensive here and the toys broke aft...,B07RBYJN37,B08N5QKX1Y,AHWBCULLVSL23PERUQAH7UQFGGTA,2022-11-14 21:14:41.618,4,True,Subscription Boxes
28,4.0,Not Bad,The second box I received was better than the ...,B08W59Z9WW,B08W5BSH6V,AHWQI5RDKU3AHHGKAKNLB6BMQSFQ,2022-08-22 12:40:11.445,2,True,Subscription Boxes
36,3.0,Don't waste your money,These items might be for little little kids (6...,B07MFQFKGD,B07MFQFKGD,AEMF2RNAUAQHTZ55AZF4KKO5AK6A,2022-08-12 15:17:01.122,0,True,Subscription Boxes
...,...,...,...,...,...,...,...,...,...,...
16187,2.0,Not worth the price.,I was very disappointed with the box I receive...,B07M8NTKKD,B07M8DC4WP,AHJLH2SKXOM4I42I6DGOEI3F2WQA,2023-08-07 02:29:14.783,2,True,Subscription Boxes
16192,3.0,As a vet,Man do I love supporting veteran run businesse...,B07WT5CCKH,B07WT5CCKH,AGRY5FEASZMNMQ4JXNQ6ZFKBZLUA,2022-05-17 01:54:36.767,0,True,Subscription Boxes
16194,4.0,Everything in it is %100 worth it,Love it,B0BM23TXKZ,B0BM23TXKZ,AGTMKQNOKNXEUUEPBP7QRKBKLDVQ,2023-08-24 02:56:45.396,0,True,Subscription Boxes
16208,5.0,Great,Great,B09R3MR7DN,B09R3MR7DN,AGJ7BPHW23T4H67J753LPUBRP7YA,2022-09-22 18:55:16.814,0,True,Subscription Boxes


In [ ]:
#exported csv will have null value even after export

### Item

- Use Chunking Method


In [6]:
# Define the categories
categories = [
    # "Digital_Music", - Done
    # "Magazine_Subscriptions", - Done
    # "Musical_Instruments",  - Done
    # "Video_Games" - Done
     "Sports_and_Outdoors",
    # "Toys_and_Games",  - Done
    # "Movies_and_TV", - Done
    # "CDs_and_Vinyl"  - Done
]

In [8]:
# Function to process and clean a chunk of data
def process_chunk(chunk, category):
    chunk = pd.DataFrame(chunk)

    # Remove some columns
    chunk = chunk.drop(["images", "videos", "categories", "bought_together", "subtitle", "author"], axis=1)

    # Rename columns
    chunk = chunk.rename(columns={
        'title': 'item_title',
        'average_rating': 'item_average_rating',
        'rating_number': 'item_rating_number'
    })

    # Drop missing values
    chunk = chunk.dropna()

    return chunk

In [9]:
# Function to load and clean dataset in chunks
def load_and_clean_category_in_chunks(category, chunk_size=100000):
    # Load the dataset
    dataset = load_dataset(
        "McAuley-Lab/Amazon-Reviews-2023",
        f"raw_meta_{category}",
        trust_remote_code=True,
    )

    # Determine available splits
    available_splits = list(dataset.keys())
    print(f"Available splits for {category}: {available_splits}")

    # Use the first available split (assuming it has data)
    split_name = available_splits[0]

    # Initialize an empty list to store processed chunks
    processed_chunks = []
    chunk_files = []

    # Process the dataset in chunks with progress bar
    for start in tqdm(range(0, len(dataset[split_name]), chunk_size), desc=f"Processing {category}"):
        end = start + chunk_size
        chunk = dataset[split_name][start:end]
        processed_chunk = process_chunk(chunk, category)
        processed_chunks.append(processed_chunk)

        # Save intermediate chunk to CSV to avoid losing progress
        chunk_file = f'processed_chunk_{category}_{start}_{end}.csv'
        processed_chunk.to_csv(chunk_file, index=False)
        chunk_files.append(chunk_file)
        print(f"Processed and saved chunk {start} to {end}")

    # Combine all processed chunks into a single DataFrame
    combined_df = pd.concat(processed_chunks, ignore_index=True)

    # Save the combined processed dataset to a CSV file
    combined_file = f'{category}_items_processed.csv'
    drive_path = '/content/drive/MyDrive/Amazon_Reviews_2023/Items/'
    combined_df.to_csv(drive_path + combined_file, index=False)

    # Delete temporary chunk files
    for chunk_file in chunk_files:
        os.remove(chunk_file)
        print(f"Deleted temporary file: {chunk_file}")

    return combined_df

In [10]:
# Loop over each category and process - 33 mins for  "Toys_and_Games", "Movies_and_TV", "CDs_and_Vinyl"
for category in categories:
    df = load_and_clean_category_in_chunks(category)
    print(f"Completed processing for category: {category}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating full split: 0 examples [00:00, ? examples/s]

Available splits for Sports_and_Outdoors: ['full']


Processing Sports_and_Outdoors:   6%|▋         | 1/16 [00:16<04:02, 16.17s/it]

Processed and saved chunk 0 to 100000


...

Processing Sports_and_Outdoors:  94%|█████████▍| 15/16 [04:26<00:17, 17.14s/it]

Processed and saved chunk 1400000 to 1500000


Processing Sports_and_Outdoors: 100%|██████████| 16/16 [04:40<00:00, 17.54s/it]

Processed and saved chunk 1500000 to 1600000


Deleted temporary file: processed_chunk_Sports_and_Outdoors_0_100000.csv
Deleted temporary file: processed_chunk_Sports_and_Outdoors_100000_200000.csv
Deleted temporary file: processed_chunk_Sports_and_Outdoors_200000_300000.csv
Deleted temporary file: processed_chunk_Sports_and_Outdoors_300000_400000.csv
Deleted temporary file: processed_chunk_Sports_and_Outdoors_400000_500000.csv
Deleted temporary file: processed_chunk_Sports_and_Outdoors_500000_600000.csv
Deleted temporary file: processed_chunk_Sports_and_Outdoors_600000_700000.csv
Deleted temporary file: processed_chunk_Sports_and_Outdoors_700000_800000.csv
Deleted temporary file: processed_chunk_Sports_and_Outdoors_800000_900000.csv
Deleted temporary file: processed_chunk_Sports_and_Outdoors_900000_1000000.csv
Deleted temporary file: processed_chunk_Sports_and_Outdoors_1000000_1100000.csv
Deleted temporary file: processed_chunk_Sports_and_Outdoors_1100000_1200000.csv
Deleted temporary file: processed_chunk_Sports_and_Outdoors_1200